In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-05-09 00:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-05-08 19:22:37,773 INFO: Initializing external client
2025-05-08 19:22:37,774 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 19:22:39,289 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633


In [6]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 


In [7]:
df

,pickup_location_id,pickup_hour,predicted_demand
0,HB407,2025-05-08 02:00:00+00:00,0.0
1,JC072,2025-05-08 02:00:00+00:00,0.0
2,JC009,2025-05-08 02:00:00+00:00,-0.0
3,JC053,2025-05-08 02:00:00+00:00,0.0
4,HB101,2025-05-08 02:00:00+00:00,0.0
...,...,...,...
277,JC032,2025-05-09 00:00:00+00:00,0.0
278,JC080,2025-05-09 00:00:00+00:00,0.0
279,JC081,2025-05-09 00:00:00+00:00,0.0
280,HB408,2025-05-09 00:00:00+00:00,1.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  282 non-null    object                 
 1   pickup_hour         282 non-null    datetime64[us, Etc/UTC]
 2   predicted_demand    282 non-null    float64                
dtypes: datetime64[us, Etc/UTC](1), float64(1), object(1)
memory usage: 6.7+ KB


In [9]:
from datetime import datetime, timezone, timedelta
import pandas as pd
from src.inference import get_feature_store
import src.config as config

def fetch_next_hour_predictions():
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    start_time = pd.Timestamp(next_hour.date(), tz="UTC")
    end_time = start_time + timedelta(days=1)

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # Read predictions for the full day
    df = fg.filter(
        (fg.pickup_hour >= start_time) & 
        (fg.pickup_hour < end_time)
    ).read()

    # Filter the predictions for the exact next hour
    df_hour = df[df['pickup_hour'] == pd.Timestamp(next_hour)]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {start_time} to {end_time}")
    print(f"Found {len(df_hour)} records")

    return df_hour


# Example usage:
predictions = fetch_next_hour_predictions()

2025-05-08 19:22:42,453 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 19:22:42,460 INFO: Initializing external client
2025-05-08 19:22:42,461 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 19:22:43,323 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.45s) 
Current UTC time: 2025-05-08 23:22:42.453674+00:00
Next hour: 2025-05-09 00:00:00+00:00
Querying for date range: 2025-05-09 00:00:00+00:00 to 2025-05-10 00:00:00+00:00
Found 92 records


In [10]:
now = datetime.now(timezone.utc)

In [11]:
predictions

,pickup_location_id,pickup_hour,predicted_demand
95,JC082,2025-05-09 00:00:00+00:00,1.0
96,JC109,2025-05-09 00:00:00+00:00,1.0
97,JC095,2025-05-09 00:00:00+00:00,0.0
98,5282.02,2025-05-09 00:00:00+00:00,-0.0
99,HB611,2025-05-09 00:00:00+00:00,1.0
...,...,...,...
182,JC032,2025-05-09 00:00:00+00:00,0.0
183,JC080,2025-05-09 00:00:00+00:00,0.0
184,JC081,2025-05-09 00:00:00+00:00,0.0
185,HB408,2025-05-09 00:00:00+00:00,1.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  282 non-null    object                 
 1   pickup_hour         282 non-null    datetime64[us, Etc/UTC]
 2   predicted_demand    282 non-null    float64                
dtypes: datetime64[us, Etc/UTC](1), float64(1), object(1)
memory usage: 6.7+ KB


In [13]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [14]:
dt = current_date.ceil('h')


In [15]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
#query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-05-08 19:22:45,641 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 19:22:45,648 INFO: Initializing external client
2025-05-08 19:22:45,649 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 19:22:46,463 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.52s) 


In [16]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 


,pickup_location_id,pickup_hour,predicted_demand


In [17]:
results

,pickup_location_id,pickup_hour,predicted_demand
0,HB407,2025-05-08 02:00:00+00:00,0.0
1,JC072,2025-05-08 02:00:00+00:00,0.0
2,JC009,2025-05-08 02:00:00+00:00,-0.0
3,JC053,2025-05-08 02:00:00+00:00,0.0
4,HB101,2025-05-08 02:00:00+00:00,0.0
...,...,...,...
277,JC032,2025-05-09 00:00:00+00:00,0.0
278,JC080,2025-05-09 00:00:00+00:00,0.0
279,JC081,2025-05-09 00:00:00+00:00,0.0
280,HB408,2025-05-09 00:00:00+00:00,1.0


In [18]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [19]:
predictions = fetch_next_hour_predictions()

2025-05-08 19:22:50,210 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 19:22:50,216 INFO: Initializing external client
2025-05-08 19:22:50,216 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 19:22:51,100 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Current UTC time: 2025-05-08 23:22:50.210368+00:00
Next hour: 2025-05-09 00:00:00+00:00
Found 92 records


In [20]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-05-08 19:22:51,884 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 19:22:51,889 INFO: Initializing external client
2025-05-08 19:22:51,890 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 19:22:52,749 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633


In [21]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.91s) 


In [22]:
df["pickup_hour"].max()

Timestamp('2025-05-09 00:00:00+0000', tz='Etc/UTC')

In [23]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-05-09 00:00:00+00:00


In [24]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,pickup_hour,predicted_demand
190,JC082,2025-05-09 00:00:00+00:00,1.0
191,JC109,2025-05-09 00:00:00+00:00,1.0
192,JC095,2025-05-09 00:00:00+00:00,0.0
193,5282.02,2025-05-09 00:00:00+00:00,-0.0
194,HB611,2025-05-09 00:00:00+00:00,1.0
...,...,...,...
277,JC032,2025-05-09 00:00:00+00:00,0.0
278,JC080,2025-05-09 00:00:00+00:00,0.0
279,JC081,2025-05-09 00:00:00+00:00,0.0
280,HB408,2025-05-09 00:00:00+00:00,1.0
